<a href="https://colab.research.google.com/github/ValhallaAMB/Gesture-Recognition/blob/main/custom_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**NOTE:**
You might have to run the first cell and then restart the session by pressing `Ctrl+m+.`, then re-run the cell. It should work after that

In [ ]:
# Install Dependencies
!pip install --upgrade pip
!pip install mediapipe-model-maker

In [ ]:
# Import Dependencies
from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import gesture_recognizer

%matplotlib inline
import matplotlib.pyplot as plt

Mount your google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Add the path to the zipped folder and unzip it.

In [ ]:
# path structure: /content/drive/MyDrive/path-to-dataset

!unzip /content/drive/MyDrive/...
dataset_path = "Data" # The name of your folder on the sidebar (<-- left sidebar)

Print the folders that we have

In [5]:
print(dataset_path)
folder_labels = []
for i in os.listdir(dataset_path):
  print(i)
  if os.path.isdir(os.path.join(dataset_path, i)):
    folder_labels.append(i)
print(len(folder_labels))

Data
I
L
W
V
E
Y
R
B
D
K
U
M
A
G
O
Z
C
N
P
H
J
S
F
Q
T
None
X
27


Loading the dataset into the gesture recogniser model and splitting the data into sections.

In [6]:
# Load dataset into gesture recognizer
data = gesture_recognizer.Dataset.from_folder(
    dirname=dataset_path,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)
# Split the data into training and validation sets
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/gesture_embedder


Setting up the hyperparameters (needs tinkering) and training the model

In [ ]:
# Set hyperparameters, customizable parameters which affect model accuracy
hparams = gesture_recognizer.HParams(learning_rate=0.001, epochs=10, batch_size=32, gamma=0, lr_decay=0.995, shuffle=True, export_dir="exported_model")

# Additional ustomizable parameter that affects accuracy
model_options = gesture_recognizer.ModelOptions(dropout_rate=0.1, layer_widths=[128, 64])

# Gathering these parameters into GestureRecognizerOptions
options = gesture_recognizer.GestureRecognizerOptions(model_options=model_options, hparams=hparams)
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Evaluating the accuracy and loss of the model

In [ ]:
# loss, acc = model.evaluate(test_data, batch_size=1)
# print(f"Test loss:{loss}, Test accuracy:{acc}")

metric_functions = model._metric_functions + [tf.keras.metrics.F1Score(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
model._model.compile(
    optimizer=model._optimizer,
    loss=model._loss_function,
    metrics=metric_functions,
)

metrics = model.evaluate(test_data)
print(metrics)

In [ ]:
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Predict on test data
predictions = []
true_labels = []
for ds in test_data.gen_tf_dataset(batch_size=32):
    x_batch, y_batch = ds
    y_pred = model._model.predict(x_batch)
    predictions.extend(np.argmax(y_pred, axis=1))
    true_labels.extend(np.argmax(y_batch.numpy(), axis=1))

In [ ]:
# Generate and plot confusion matrix
cm = confusion_matrix(true_labels, predictions)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Plot the heatmap using GeeksforGeeks style https://www.geeksforgeeks.org/confusion-matrix-machine-learning/
plt.figure(figsize=(20, 18))
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='YlGnBu',
            xticklabels=folder_labels, yticklabels=folder_labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Normalized Confusion Matrix - Gesture Recognition Model')
plt.xticks(rotation=45)
plt.yticks(rotation=45)
plt.tight_layout()
plt.show()

# print("True labels collected:", len(true_labels))
# print("Predictions collected:", len(predictions))
# print("Unique true labels:", np.unique(true_labels))
# print("Unique predicted labels:", np.unique(predictions))
# print("Confusion matrix shape:", cm.shape)
# print("Confusion matrix content:\n", cm)

Exporting the model to download a `.task` file

In [ ]:
model.export_model()
!ls exported_model

Saving the model onto your local machine.

In [ ]:
files.download('exported_model/gesture_recognizer.task')

This **removes/deletes** folders. BE CAREFUL WHEN USING THIS BLOCK OF CODE

In [3]:
# !rm -rf exported_model
# !rm -rf Data